In [9]:
# Importing libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [10]:
# Load dataset
og_df = pd.read_csv("data.csv")
og_df.head()

,ID,air_time1,disp_index1,gmrt_in_air1,gmrt_on_paper1,max_x_extension1,max_y_extension1,mean_acc_in_air1,mean_acc_on_paper1,mean_gmrt1,...,mean_jerk_in_air25,mean_jerk_on_paper25,mean_speed_in_air25,mean_speed_on_paper25,num_of_pendown25,paper_time25,pressure_mean25,pressure_var25,total_time25,class
0,id_1,5160,0.000013,120.804174,86.853334,957,6601,0.361800,0.217459,103.828754,...,0.141434,0.024471,5.596487,3.184589,71,40120,1749.278166,296102.7676,144605,P
1,id_2,51980,0.000016,115.318238,83.448681,1694,6998,0.272513,0.144880,99.383459,...,0.049663,0.018368,1.665973,0.950249,129,126700,1504.768272,278744.2850,298640,P
2,id_3,2600,0.000010,229.933997,172.761858,2333,5802,0.387020,0.181342,201.347928,...,0.178194,0.017174,4.000781,2.392521,74,45480,1431.443492,144411.7055,79025,P
3,id_4,2130,0.000010,369.403342,183.193104,1756,8159,0.556879,0.164502,276.298223,...,0.113905,0.019860,4.206746,1.613522,123,67945,1465.843329,230184.7154,181220,P
4,id_5,2310,0.000007,257.997131,111.275889,987,4732,0.266077,0.145104,184.636510,...,0.121782,0.020872,3.319036,1.680629,92,37285,1841.702561,158290.0255,72575,P


In [11]:
og_df.shape

(174, 452)

In [12]:
og_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174 entries, 0 to 173
Columns: 452 entries, ID to class
dtypes: float64(300), int64(150), object(2)
memory usage: 614.6+ KB


In [13]:
og_df.dtypes

ID                  object
air_time1            int64
disp_index1        float64
gmrt_in_air1       float64
gmrt_on_paper1     float64
                    ...   
paper_time25         int64
pressure_mean25    float64
pressure_var25     float64
total_time25         int64
class               object
Length: 452, dtype: object

In [14]:
# Check for any missing values
og_df.isnull().sum()

ID                 0
air_time1          0
disp_index1        0
gmrt_in_air1       0
gmrt_on_paper1     0
                  ..
paper_time25       0
pressure_mean25    0
pressure_var25     0
total_time25       0
class              0
Length: 452, dtype: int64

In [19]:
# Rename values in 'class' column
# og_df['class'] = og_df['class'].map({'P' : 0, 'H' : 1})
# og_df = og_df.drop(columns=['ID'], axis=1, inplace=True)
# corr_matrix = og_df.corr()

TypeError: 'NoneType' object is not subscriptable

In [ ]:
# Select relevant characteristics: MJP, MJA, MSP, MSA, TT, PM, Class(label: patient or healthy)
trial1 = pd.DataFrame(og_df[['mean_jerk_on_paper1', 'mean_jerk_in_air1', 'mean_speed_on_paper1', 
                      'mean_speed_in_air1', 'total_time1', 'pressure_mean1', 'class']])

trial2 = pd.DataFrame(og_df[['mean_jerk_on_paper2', 'mean_jerk_in_air2', 'mean_speed_on_paper2', 
                             'mean_speed_in_air2', 'total_time2', 'pressure_mean2', 'class']])

trial6 = pd.DataFrame(og_df[['mean_jerk_on_paper6', 'mean_jerk_in_air6', 'mean_speed_on_paper6', 
                      'mean_speed_in_air6', 'total_time6', 'pressure_mean6', 'class']])

trial1.columns = trial2.columns = trial6.columns = ['MJP', 'MJA', 'MSP', 'MSA', 'TT', 'PM', 'Class']